In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/JudithAbecassis/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
gotv = pd.read_excel('../problem_set_1/gotv_individual.xlsx')

In [3]:
gotv.head()

,treatment,hh_id,hh_size,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,p2004_mean,g2004_mean,voted
0,Civic Duty,1.0,2,male,1941,1,1,1,0,1,0,0.095238,0.857143,0
1,Civic Duty,1.0,2,female,1947,1,1,1,0,1,0,0.095238,0.857143,0
2,Hawthorne,2.0,3,male,1951,1,1,1,0,1,0,0.047619,0.857143,1
3,Hawthorne,2.0,3,female,1950,1,1,1,0,1,0,0.047619,0.857143,1
4,Hawthorne,2.0,3,female,1982,1,1,1,0,1,0,0.047619,0.857143,1


In [4]:
previous_elections = ['g2000', 'p2000', 'g2002', 'p2002', 'p2004']
gotv = gotv.assign(num_voted=gotv[previous_elections].sum(axis=1))

Let's check if the new column `num_voted` has the right properties.

In [5]:
gotv.num_voted.value_counts()

3    136031
2     80436
4     65852
1     26060
0     21784
5     13921
Name: num_voted, dtype: int64

In [6]:
gotv = gotv.assign(id=gotv.index)
final_cols = ['id', 'hh_id', 'hh_size', 'num_voted', 'voted', 'treatment']
sub_gotv = gotv[gotv.treatment.isin([' Control', ' Neighbors'])][final_cols]
gotv_house = sub_gotv.groupby(['hh_id', 'treatment'])['hh_size', 'num_voted', 'voted'].mean().reset_index(level=1)

In [7]:
gotv_house.head(10)

,treatment,hh_size,num_voted,voted
hh_id,,,,
3.0,Control,3.0,2.00,0.666667
4.0,Control,2.0,2.00,0.000000
5.0,Control,1.0,3.00,1.000000
7.0,Control,1.0,2.00,0.000000
8.0,Control,2.0,4.50,1.000000
10.0,Control,1.0,3.00,1.000000
11.0,Neighbors,1.0,3.00,1.000000
13.0,Control,2.0,4.00,1.000000
14.0,Control,4.0,1.75,0.000000


In [8]:
gotv_house = gotv_house.assign(num_voted=np.round(gotv_house.num_voted))

In [9]:
# checking
gotv_house.head(10)

,treatment,hh_size,num_voted,voted
hh_id,,,,
3.0,Control,3.0,2.0,0.666667
4.0,Control,2.0,2.0,0.000000
5.0,Control,1.0,3.0,1.000000
7.0,Control,1.0,2.0,0.000000
8.0,Control,2.0,4.0,1.000000
10.0,Control,1.0,3.0,1.000000
11.0,Neighbors,1.0,3.0,1.000000
13.0,Control,2.0,4.0,1.000000
14.0,Control,4.0,2.0,0.000000


In [10]:
# checking
gotv_house.num_voted.value_counts()

2.0    45153
3.0    38744
4.0    28926
1.0     5479
5.0     1235
0.0      462
Name: num_voted, dtype: int64

Let's now look at the number of household in each subgroup for treatment and control.

In [11]:
pd.crosstab(gotv_house.num_voted, gotv_house.treatment,
            values=gotv_house.voted, aggfunc='count')

treatment,Control,Neighbors
num_voted,,
0.0,383,79
1.0,4572,907
2.0,37602,7551
3.0,32290,6454
4.0,24138,4788
5.0,1014,221


It can be hard to interprete, let's look at the proportions in each subgroup.

In [12]:
pd.crosstab(gotv_house.num_voted, gotv_house.treatment, values=gotv_house.voted,
            aggfunc='count', normalize='columns')

treatment,Control,Neighbors
num_voted,,
0.0,0.003830,0.00395
1.0,0.045720,0.04535
2.0,0.376024,0.37755
3.0,0.322903,0.32270
4.0,0.241382,0.23940
5.0,0.010140,0.01105


We have exactly the same proportion of each subgroup in the two treatment levels.

In [13]:
res_list = []
for i in range(6):
    sub_gotv_house = gotv_house[gotv_house.num_voted==i]
    cate = sub_gotv_house[sub_gotv_house.treatment==' Neighbors'].voted.mean() - sub_gotv_house[sub_gotv_house.treatment==' Control'].voted.mean()
    var_cate = sub_gotv_house[sub_gotv_house.treatment==' Neighbors'].voted.var() / len(sub_gotv_house[sub_gotv_house.treatment==' Neighbors']) +\
        sub_gotv_house[sub_gotv_house.treatment==' Control'].voted.var() / len(sub_gotv_house[sub_gotv_house.treatment==' Control'])
    ci_low = cate - sps.norm.ppf(0.975) * np.sqrt(var_cate)
    ci_high = cate + sps.norm.ppf(0.975) * np.sqrt(var_cate)
    n_x = len(sub_gotv_house)
    res_list.append([cate, var_cate, ci_low, ci_high, n_x])
res_df = pd.DataFrame(res_list, columns=['cate', 'var_cate', "ci_low", "ci_high", "n_x"])
res_df

,cate,var_cate,ci_low,ci_high,n_x
0,0.101533,0.002522,0.003112,0.199954,462
1,0.068650,0.000175,0.042741,0.094559,5479
2,0.068592,0.000025,0.058839,0.078344,45153
3,0.097372,0.000039,0.085200,0.109544,38744
4,0.098715,0.000052,0.084532,0.112898,28926
5,0.040374,0.001253,-0.029007,0.109755,1235


We see a positive significant effect on all the groups of the same order of magnitude, except the group where the household voted at all of the previous elections, where the effect is not significant.

In [14]:
delta_hat = res_df.iloc[0].cate - res_df.iloc[5].cate
var_delta_hat = res_df.iloc[0].var_cate + res_df.iloc[5].var_cate
ci_delta = [delta_hat - sps.norm.ppf(0.975) * np.sqrt(var_delta_hat),
            delta_hat + sps.norm.ppf(0.975) * np.sqrt(var_delta_hat)]
print(delta_hat, var_delta_hat, ci_delta)

0.061158998627744876 0.003774726097855774 [-0.05925883586662052, 0.18157683312211026]


Between the most two extreme groups, i.e. people who never vote and people who always vote, we don't find a significant difference in the treatment effect. So even if we can't reject the null hypothesis of no effect in the group of people who always vote, we don't have enough power to reject the null hypothesis of a different treatment effect among the subgroups. This may seem paradoxical, but what we can say intuitively is that we have enough statistical power to distinguidh the effect in the people who never vote from zero, but not from 0.04. To increase our statistical power, we can consider a larger sample size. We discuss something related in question 4 (we see the effect of having a smaller sample size in each group).

Suppose the overall sample size is $N_c$ and $N_t$, as the number of groups x increase, we can assume that the sample size for each group is evenly spread as $N_c/x$ or $N_t/x$. Recall that in our calculation of confidence interval, we are taking a factor of group sizes by $\sqrt(n)$. Thus, if we increase the number of subgroup from 1 to 16, the confidence interval we are getting could increase by $sqrt(16) = 4$, so the new CI could be 4 times the former one, where ATE would have to be much larger for the differences to be significant. 

(An intuitive explanation of how increasing number of subgroup/multiple hypothesis testing would lead to decrease in sample size per group, and ultimately required larger ATE to have significant difference)

In [15]:
ate = gotv_house[gotv_house.treatment==' Neighbors'].voted.mean() - \
    gotv_house[gotv_house.treatment==' Control'].voted.mean()
var_ate = gotv_house[gotv_house.treatment==' Neighbors'].voted.var() /\
    len(gotv_house[gotv_house.treatment==' Neighbors']) + \
    gotv_house[gotv_house.treatment==' Control'].voted.var() /\
    len(gotv_house[gotv_house.treatment==' Control'])
ci_ate = [ate - sps.norm.ppf(0.975) * np.sqrt(var_ate),
          ate + sps.norm.ppf(0.975) * np.sqrt(var_ate)]

print(ate, var_ate, ci_ate)

0.08478807597718824 1.1568354434982345e-05 [0.07812179086588532, 0.09145436108849116]


In [16]:
tau_block = np.sum(res_df.cate * res_df.n_x) / len(gotv_house)
tau_block

0.0849843855593769

In [17]:
var_tau_block = np.sum(res_df.var_cate * res_df.n_x**2) / len(gotv_house)**2
var_tau_block

1.110373595388372e-05

In [18]:
ci_tau_block = [tau_block - sps.norm.ppf(0.975) * np.sqrt(var_tau_block),
                tau_block + sps.norm.ppf(0.975) * np.sqrt(var_tau_block)]
print(ci_tau_block)

[0.0784533408938808, 0.091515430224873]


We observe a subtle difference between the two estimators.Probably because the stratifying variable is not that strongly correlated with the outcome. We verify experimentally that $Var(\hat{\tau}) \geq Var(\hat{\tau}_{block})$.


In [19]:
gotv_house.treatment.value_counts() % 6

 Control      3
 Neighbors    2
Name: treatment, dtype: int64

In [20]:
# we remote some units to have a number of units divi
rg = np.random.default_rng(12)
pre_control = gotv_house[gotv_house.treatment==' Control']
control = pre_control.sample(len(pre_control) - len(pre_control) % 6)
pre_neighbors = gotv_house[gotv_house.treatment==' Neighbors']
neighbors = pre_neighbors.sample(len(pre_neighbors) - len(pre_neighbors) % 6)

control_groups = list(range(6)) * int(len(control) / 6)
neighbors_groups = list(range(6)) * int(len(neighbors) / 6)

control = control.assign(groups=rg.permutation(control_groups))
neighbors = neighbors.assign(groups=rg.permutation(neighbors_groups))

new_gotv_house = pd.concat([control, neighbors])

In [21]:
control.head(10)

,treatment,hh_size,num_voted,voted,groups
hh_id,,,,,
50569.0,Control,2.0,4.0,0.000000,5
162826.0,Control,3.0,2.0,0.333333,2
93815.0,Control,2.0,4.0,1.000000,5
4043.0,Control,2.0,2.0,0.000000,3
6155.0,Control,2.0,4.0,1.000000,4
67787.0,Control,2.0,3.0,0.000000,3
127769.0,Control,1.0,2.0,0.000000,2
75987.0,Control,2.0,3.0,0.000000,5
74141.0,Control,2.0,3.0,0.000000,3


In [22]:
res_list = []
for i in range(6):
    sub_gotv_house = new_gotv_house[new_gotv_house.groups==i]
    cate = sub_gotv_house[sub_gotv_house.treatment==' Neighbors'].voted.mean() -\
        sub_gotv_house[sub_gotv_house.treatment==' Control'].voted.mean()
    var_cate = sub_gotv_house[sub_gotv_house.treatment==' Neighbors'].voted.var() / \
        len(sub_gotv_house[sub_gotv_house.treatment==' Neighbors']) +\
        sub_gotv_house[sub_gotv_house.treatment==' Control'].voted.var() /\
        len(sub_gotv_house[sub_gotv_house.treatment==' Control'])
    ci_low = cate - sps.norm.ppf(0.975) * np.sqrt(var_cate)
    ci_high = cate + sps.norm.ppf(0.975) * np.sqrt(var_cate)
    n_x = len(sub_gotv_house)
    res_list.append([cate, var_cate, ci_low, ci_high, n_x])
new_res_df = pd.DataFrame(res_list, columns=['cate', 'var_cate', "ci_low", "ci_high", "n_x"])
new_res_df

,cate,var_cate,ci_low,ci_high,n_x
0,0.067578,0.000068,0.051389,0.083767,19999
1,0.091207,0.000070,0.074787,0.107626,19999
2,0.094024,0.000070,0.077656,0.110391,19999
3,0.074438,0.000069,0.058105,0.090771,19999
4,0.097255,0.000070,0.080858,0.113652,19999
5,0.084466,0.000069,0.068202,0.100730,19999


In [23]:
new_tau_block = np.sum(new_res_df.cate * new_res_df.n_x) / len(new_gotv_house)
new_tau_block

0.08482786020397422

In [24]:
new_var_tau_block = np.sum(new_res_df.var_cate * new_res_df.n_x**2) / len(new_gotv_house)**2
new_var_tau_block

1.1567600360170545e-05

In [25]:
new_ci_tau_block = [new_tau_block - sps.norm.ppf(0.975) * np.sqrt(new_var_tau_block),
                    new_tau_block + sps.norm.ppf(0.975) * np.sqrt(new_var_tau_block)]
print(new_ci_tau_block)

[0.07816179236469732, 0.09149392804325111]
